In [ ]:
(core:getpid)

In [ ]:
(start-swank)

In [ ]:
(ql:quickload :design)

In [ ]:
(cl-jupyter-user:png-from-file "foo.png")

Extract all of the topologys from the structure

In [ ]:
(:= *be* (multiple-value-list (design.load::load-cdxml "foo.cdxml")))

In [ ]:
*be*

Classify the topologys into origin, body and cap topologys

In [ ]:
(defvar origins)
(defvar bodys)
(defvar caps)
(multiple-value-setq (origins bodys caps) (design::classify-topologys (car *be*)))

In [ ]:
(chem:get-stereoisomer-name (car (alexandria:hash-table-values origins)) 0)

In [ ]:
(:= *cap-name-map* (alexandria:plist-hash-table (list :+origin :ccap-origin :+default :ncap :-default :leub.origin)))
(:= *top-map* (let ((ht (make-hash-table))) 
                   (loop for top in (car *be*) 
                             do (setf (gethash (chem:get-name top) ht) top))
                   ht))

In [ ]:
*top-map*

In [ ]:
(:= *olig* (design::build-one-training-oligomer bodys *cap-name-map* *top-map* :test t))

In [ ]:
(design::one-round-extend-oligomer-with-caps *olig* *cap-name-map* *top-map*)

In [ ]:
(:= *oligs* (design::build-training-oligomers (car *be*) *cap-name-map*))

In [ ]:
*oligs*

In [ ]:
(chem:number-of-sequences (second *oligs*))

In [ ]:
(mapcar (lambda (x) (chem:get-monomer-index x)) (chem:monomers-as-list (second *oligs*)))

In [ ]:
(chem:current-sequence-index (second *oligs*))

In [ ]:
(:= *mol* (chem:get-molecule (second *oligs*)))

In [ ]:
(jostle *mol*)

In [ ]:
(show *mol*)

In [ ]:
(:= *agg* (chem:make-aggregate))

In [ ]:
(chem:add-molecule *agg* *mol*)

In [ ]:
(easy-gaff)

In [ ]:
(minimize *agg*)

In [ ]:
(show *agg*)

In [ ]:
1

In [ ]:
(apropos "save-cando")

In [ ]:
(save-cando *agg* "/tmp/foo.cando")

In [ ]:
(load-cando "/tmp/foo.cando")

In [ ]:
(design::find-unsatisfied-monomer-plug-pairs *olig*)

In [ ]:
(design::one-round-extend-oligomer-with-caps *olig* *cap-name-map* *top-map*)

In [ ]:
(chem:number-of-sequences *olig*)

In [ ]:
(alexandria:hash-table-values *cap-map*)

In [ ]:
(chem:coupling-name :+foo)

In [ ]:
(build
 (frame :in "N1C2[CH2]3C=O")
 (frame :origin "C1(=O2)C3")
 (cap :+origin :ccap)
 (cap :+default :ncap)
 (cap :-default :leub.origin))

In [ ]:
(:= *o0* (design:make-oligomer '(:leub.origin (:origin :ccap-origin)
                                              :default :ncap)))

In [ ]:
(design::build-all-molecules *o0*)

In [ ]:
(:= *o* (design:make-oligomer '(:leub.origin (:origin :ccap-origin) :default :leub 
                                             :default :thrb
                                             :default :leub
                                             :default :ncap)))

In [ ]:
(chem:monomers-as-list *o*)

In [ ]:
(:= *aggs* (design::build-all-molecules *o*))

In [ ]:
(:= *agg* (first *aggs*))

In [ ]:
(:= *m* (chem:build-atom-id-map *agg*))

In [ ]:
(chem:lookup-atom *m* '(0 0 2) )

In [ ]:
(:= *conf* (kin:make-conformation (list *o*)))

In [ ]:
*aggs*

In [ ]:
(:= *agg* (first *aggs*))

In [ ]:
(easy-gaff)

In [ ]:
(energy:minimize *agg*)

In [ ]:
(show *agg*)

In [ ]:
*m*

In [ ]:
(kin:walk (kin:get-atom-tree *conf*) 
          (lambda (x) 
                  (print (chem:lookup-atom *m* (kin:get-atom-id x)))))

In [ ]:
(graphviz-dot 'design.graphviz-draw-joint-template:draw-joint-template *jt* "temp")

In [ ]:
(:= *olig* (core:make-cxx-object 'chem:oligomer))

In [ ]:
(apropos "atom-id")

In [ ]:
*olig*

In [ ]:
(design.oligomer::load-cdxml "foo.cdxml")

In [ ]:
(print "Hello world")

In [ ]:
(ql:quickload :cl-bqplot)

In [ ]:
(defun plot (fn &key (xrange (list 0.0 1.0)) (divisions 100))
   (let ((xarray (make-array divisions :element-type 'single-float))
         (yarray (make-array divisions :element-type 'single-float))
         (figure (bqplot::figure)))
        (loop for idx below divisions
              for x = (+ (* (- (second xrange) (first xrange))
                            (/ (float idx) (float divisions)))
                         (first xrange))
              do (setf (aref xarray idx) (float x 1s0))
               do (setf (aref yarray idx) (float (funcall fn x) 1s0)))
        (bqplot::scatter xarray yarray)
        figure))